## Informe de Extracción, Transformación y Carga de Datos (ETL): homicidios.xlsx

En este apartado, llevaremos a cabo el proceso de Extracción, Transformación y Carga (ETL) del archivo excel: "homicidios.xlsx", exploraremos cada hoja de trabajo (HECHOS y VICTIMAS) para lograr su comprensión y preparar los datos para una furutas exploraciones y análsis que puedan llevar hacia la acción de mitigar las pérdidas humanas en las carreteras de la Ciudad de Buenos Aires.

Empezaremos importando las bibliotecas necesarias para el trabajo, llevaremos a cabo técnicas de depuración y procesamiento, para finalmente almacenar los datos transformados en un archivo csv.

### 1. Importación de las bibliotecas necesarias

In [1]:
# Pandas para el análisis de datos tabulares
import pandas as pd

# NumPy proporciona soporte para arreglos y matrices multidimensionales
import numpy as np

# Seaborn para la visualización de datos mediante gráficos estadísticos
import seaborn as sns

# Matplotlib para la creación de gráficos y visualizaciones
import matplotlib.pyplot as plt

# Funciones útiles
from functions import cargar_archivos_excel
from functions import analizar_valores_sd
from functions import data_cleaning

### 2. Carga del archivo

In [2]:
# Ruta del Archivo
ruta = "../Datasets/homicidios.xlsx"

In [3]:
# Llamamos a la función "cargar_archivos_excel"
datos_homicidios = cargar_archivos_excel(ruta, ["HECHOS", "VICTIMAS"])

### 3. Exploración y Limpieza de Datos

#### 3.1. Dataset "HECHOS"

In [4]:
df_hechos = datos_homicidios["HECHOS"]

In [5]:
# Mostramos las primeras 3 filas del DataFrame
df_hechos.head(3)

,ID,N_VICTIMAS,FECHA,AAAA,MM,DD,HORA,HH,LUGAR_DEL_HECHO,TIPO_DE_CALLE,...,Altura,Cruce,Dirección Normalizada,COMUNA,XY (CABA),pos x,pos y,PARTICIPANTES,VICTIMA,ACUSADO
0,2016-0001,1,2016-01-01,2016,1,1,04:00:00,4,AV PIEDRA BUENA Y AV FERNANDEZ DE LA CRUZ,AVENIDA,...,NaN,"FERNANDEZ DE LA CRUZ, F., GRAL. AV.","PIEDRA BUENA AV. y FERNANDEZ DE LA CRUZ, F., G...",8,Point (98896.78238426 93532.43437792),-58.47533969,-34.68757022,MOTO-AUTO,MOTO,AUTO
1,2016-0002,1,2016-01-02,2016,1,2,01:15:00,1,AV GRAL PAZ Y AV DE LOS CORRALES,GRAL PAZ,...,NaN,DE LOS CORRALES AV.,"PAZ, GRAL. AV. y DE LOS CORRALES AV.",9,Point (95832.05571093 95505.41641999),-58.50877521,-34.66977709,AUTO-PASAJEROS,AUTO,PASAJEROS
2,2016-0003,1,2016-01-03,2016,1,3,07:00:00,7,AV ENTRE RIOS 2034,AVENIDA,...,2034.0,NaN,ENTRE RIOS AV. 2034,1,Point (106684.29090040 99706.57687843),-58.39040293,-34.63189362,MOTO-AUTO,MOTO,AUTO


In [6]:
# Revisamos las información general del DataFrame
df_hechos.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 696 entries, 0 to 695
Data columns (total 21 columns):
 #   Column                 Non-Null Count  Dtype         
---  ------                 --------------  -----         
 0   ID                     696 non-null    object        
 1   N_VICTIMAS             696 non-null    int64         
 2   FECHA                  696 non-null    datetime64[ns]
 3   AAAA                   696 non-null    int64         
 4   MM                     696 non-null    int64         
 5   DD                     696 non-null    int64         
 6   HORA                   696 non-null    object        
 7   HH                     696 non-null    object        
 8   LUGAR_DEL_HECHO        696 non-null    object        
 9   TIPO_DE_CALLE          696 non-null    object        
 10  Calle                  695 non-null    object        
 11  Altura                 129 non-null    float64       
 12  Cruce                  525 non-null    object        
 13  Direc

Tras verificar el DataFrame, identificamos que los valores "SD" corresponden a "Sin Dato", por lo que se tomó la decisión de cambiar estos valores a NaN para estandarizar.

In [7]:
# Observamos la presencia de datos "SD" en el DataFrame
df_hechos.eq("SD").sum()

ID                        0
N_VICTIMAS                0
FECHA                     0
AAAA                      0
MM                        0
DD                        0
HORA                      1
HH                        1
LUGAR_DEL_HECHO           1
TIPO_DE_CALLE             0
Calle                     0
Altura                    0
Cruce                     0
Dirección Normalizada     0
COMUNA                    0
XY (CABA)                 0
pos x                     0
pos y                     0
PARTICIPANTES             0
VICTIMA                   9
ACUSADO                  23
dtype: int64

In [8]:
# Llamamos a la función "analizar_valores_sd"
analizar_valores_sd(df_hechos)

,Columna,Cantidad de SD,Porcentaje de SD
6,HORA,1,0.143678
7,HH,1,0.143678
8,LUGAR_DEL_HECHO,1,0.143678
19,VICTIMA,9,1.293103
20,ACUSADO,23,3.304598


In [9]:
# Reemplazamos "SD" por NaN en todo el DataFrame
df_hechos.replace(["SD","sd"], np.nan, inplace=True)

Preparamos los datos que usaremos al llamar a la función "data_cleaning".

In [10]:
# Columnas que pasaremos a minúcula
columns_to_lower = ["TIPO_DE_CALLE", "VICTIMA", "ACUSADO"]

# Columnas que eliminaremos
columns_to_drop = ["AAAA", "MM", "DD", "HORA", "LUGAR_DEL_HECHO", "Calle", "Altura",
                   "Cruce", "Dirección Normalizada", "XY (CABA)", "PARTICIPANTES"]

# Columnas que renombraremos
rename_dict = {"ID":"id_siniestro", "N_VICTIMAS":"nro_victimas", "FECHA":"fecha", "HH":"franja_hora",
               "TIPO_DE_CALLE":"tipo_calle", "COMUNA":"comuna", "pos x":"longitud", "pos y":"latitud",
               "VICTIMA":"vehiculo_victima", "ACUSADO":"vehiculo_acusado"}

# Columnas que convertiremos a entero
columns_to_int = ["nro_victimas", "franja_hora", "comuna"]

# Columnas que categorizaremos
columns_to_categorize = ["franja_hora", "tipo_calle", "comuna", "vehiculo_victima", "vehiculo_acusado"]

In [11]:
# Llamamos a la función "data_cleaning"

df_hechos_cleaned = data_cleaning(df_hechos,
                                    strip_spaces=True, # Eliminar espacios en blanco
                                    lowercase_columns=columns_to_lower, # Convertir a minúsculas                                    
                                    drop_columns=columns_to_drop,  # Eliminar columnas
                                    rename_columns=rename_dict,  # Renombrar columnas
                                    convert_to_int_columns=columns_to_int, #Conversión entero
                                    categorize_columns=columns_to_categorize # Categorizar columnas
                                   )

In [12]:
# Observamos el resultado
df_hechos_cleaned.head(5)

,id_siniestro,nro_victimas,fecha,franja_hora,tipo_calle,comuna,longitud,latitud,vehiculo_victima,vehiculo_acusado
0,2016-0001,1,2016-01-01,4,avenida,8,-58.47533969,-34.68757022,moto,auto
1,2016-0002,1,2016-01-02,1,gral paz,9,-58.50877521,-34.66977709,auto,pasajeros
2,2016-0003,1,2016-01-03,7,avenida,1,-58.39040293,-34.63189362,moto,auto
3,2016-0004,1,2016-01-10,0,avenida,8,-58.46503904,-34.68092974,moto,NaN
4,2016-0005,1,2016-01-21,5,avenida,1,-58.38718297,-34.62246630,moto,pasajeros


Se detectó que en el DataFrame existen registros con valor 0 en "comuna", son los siguientes:

In [13]:
df_hechos_cleaned[df_hechos_cleaned["comuna"] == 0]

,id_siniestro,nro_victimas,fecha,franja_hora,tipo_calle,comuna,longitud,latitud,vehiculo_victima,vehiculo_acusado
119,2016-0151,1,2016-11-18,20,calle,0,.,.,peaton,NaN
139,2016-0174,1,2016-12-27,0,autopista,0,.,.,NaN,NaN


Para este registro "2016-0174" se tenía la calle y, con ésta logramos obtener comuna, longitud y latitud . Para el caso del registro "2016-0151" tuvimos que buscar el siniestro más cercano "2016-0155" que haya ocurrido también en una calle y le imputamos el mismo valor a las columnas "comuna", "longitud" y "latitud".

In [14]:
df_hechos_cleaned.loc[df_hechos_cleaned["id_siniestro"] == "2016-0151", "comuna"] = 10
df_hechos_cleaned.loc[df_hechos_cleaned["id_siniestro"] == "2016-0174", "comuna"] = 7

Ahora verificaremos aquellos registros que no contengan valores en las columnas "longitud" y "latitud"

In [15]:
# Convertimos a NaN los datos en donde "longitud" y "latitud" estén vacíos
df_hechos_cleaned["longitud"] = pd.to_numeric(df_hechos_cleaned["longitud"], errors="coerce")
df_hechos_cleaned["latitud"] = pd.to_numeric(df_hechos_cleaned["latitud"], errors="coerce")

In [16]:
# Observamos los registros con datos faltantes en "longitud" y "latitud"
df_hechos_cleaned.loc[pd.isna(df_hechos_cleaned["longitud"]) & pd.isna(df_hechos_cleaned["latitud"])]

,id_siniestro,nro_victimas,fecha,franja_hora,tipo_calle,comuna,longitud,latitud,vehiculo_victima,vehiculo_acusado
38,2016-0052,1,2016-04-20,20,autopista,13,NaN,NaN,moto,NaN
106,2016-0136,1,2016-10-25,0,autopista,4,NaN,NaN,moto,cargas
119,2016-0151,1,2016-11-18,20,calle,10,NaN,NaN,peaton,NaN
139,2016-0174,1,2016-12-27,0,autopista,7,NaN,NaN,NaN,NaN
176,2017-0042,1,2017-04-10,9,gral paz,14,NaN,NaN,moto,cargas
180,2017-0050,2,2017-04-28,11,autopista,9,NaN,NaN,moto,cargas
181,2017-0051,1,2017-05-01,3,autopista,7,NaN,NaN,auto,auto
256,2017-0140,1,2017-11-19,23,autopista,4,NaN,NaN,moto,pasajeros
313,2018-0039,1,2018-04-21,22,autopista,14,NaN,NaN,peaton,auto
546,2020-0026,1,2020-05-17,6,autopista,14,NaN,NaN,moto,objeto fijo


Buscamos en la siguiente web, para obtener la información necesaria:
https://www.comollegara.com/direccion-a-lat-long.html?lugar=autopista%2025%20de%20mayo%20buenos%20aires según los datos del DataFrame inicial.

In [17]:
# Definimos los registros que deseamos reemplazar
registros_a_reemplazar = [
    {"id_siniestro": "2016-0174", "comuna": 7, "longitud": -58.369529920, "latitud": -34.622472560},
    {"id_siniestro": "2016-0151", "comuna": 10, "longitud": -58.52756365, "latitud": -34.62772274},
    {"id_siniestro": "2018-0039", "comuna": 14, "longitud": -58.457579360, "latitud": -34.538957930},
    {"id_siniestro": "2020-0026", "comuna": 14, "longitud": -58.457579360, "latitud": -34.538957930}, 
    {"id_siniestro": "2017-0042", "comuna": 14, "longitud": -58.457579360, "latitud": -34.538957930}, 
    {"id_siniestro": "2017-0050", "comuna": 9, "longitud": -58.481800300, "latitud": -34.647005340},
    {"id_siniestro": "2017-0051", "comuna": 7, "longitud": -58.479447760, "latitud": -34.678121010},     
    {"id_siniestro": "2017-0140", "comuna": 4, "longitud": -58.380781350, "latitud": -34.622364700},
    {"id_siniestro": "2020-0039", "comuna": 9, "longitud": -58.483096200, "latitud": -34.671405150},   
    {"id_siniestro": "2021-0023", "comuna": 4, "longitud": -58.364905600, "latitud": -34.622974220},
    {"id_siniestro": "2016-0052", "comuna": 13, "longitud": -58.457579360, "latitud": -34.538957930},  
    {"id_siniestro": "2016-0136", "comuna": 4, "longitud": -58.363401550, "latitud": -34.624371200},
]

In [18]:
# Iteramos sobre los registros y realizamos el reemplazo
for registro in registros_a_reemplazar:
    id_siniestro = registro["id_siniestro"]
    comuna = registro["comuna"]
    longitud = registro["longitud"]
    latitud = registro["latitud"]

    # Utilizamos loc para reemplazar los valores específicos
    df_hechos_cleaned.loc[df_hechos_cleaned["id_siniestro"] == id_siniestro, "comuna"] = comuna
    df_hechos_cleaned.loc[df_hechos_cleaned["id_siniestro"] == id_siniestro, "longitud"] = longitud
    df_hechos_cleaned.loc[df_hechos_cleaned["id_siniestro"] == id_siniestro, "latitud"] = latitud

In [19]:
# Mostramos el resultado
df_hechos_cleaned.head()

,id_siniestro,nro_victimas,fecha,franja_hora,tipo_calle,comuna,longitud,latitud,vehiculo_victima,vehiculo_acusado
0,2016-0001,1,2016-01-01,4,avenida,8,-58.475340,-34.687570,moto,auto
1,2016-0002,1,2016-01-02,1,gral paz,9,-58.508775,-34.669777,auto,pasajeros
2,2016-0003,1,2016-01-03,7,avenida,1,-58.390403,-34.631894,moto,auto
3,2016-0004,1,2016-01-10,0,avenida,8,-58.465039,-34.680930,moto,NaN
4,2016-0005,1,2016-01-21,5,avenida,1,-58.387183,-34.622466,moto,pasajeros


#### 3.2. Dataset "VICTIMAS"

In [20]:
df_victimas = datos_homicidios["VICTIMAS"]

In [21]:
# Mostramos las primeras 5 filas del DataFrame
df_victimas.head()

,ID_hecho,FECHA,AAAA,MM,DD,ROL,VICTIMA,SEXO,EDAD,FECHA_FALLECIMIENTO
0,2016-0001,2016-01-01,2016,1,1,CONDUCTOR,MOTO,MASCULINO,19,2016-01-01 00:00:00
1,2016-0002,2016-01-02,2016,1,2,CONDUCTOR,AUTO,MASCULINO,70,2016-01-02 00:00:00
2,2016-0003,2016-01-03,2016,1,3,CONDUCTOR,MOTO,MASCULINO,30,2016-01-03 00:00:00
3,2016-0004,2016-01-10,2016,1,10,CONDUCTOR,MOTO,MASCULINO,18,SD
4,2016-0005,2016-01-21,2016,1,21,CONDUCTOR,MOTO,MASCULINO,29,2016-02-01 00:00:00


In [22]:
# Revisamos las información general del DataFrame
df_victimas.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 717 entries, 0 to 716
Data columns (total 10 columns):
 #   Column               Non-Null Count  Dtype         
---  ------               --------------  -----         
 0   ID_hecho             717 non-null    object        
 1   FECHA                717 non-null    datetime64[ns]
 2   AAAA                 717 non-null    int64         
 3   MM                   717 non-null    int64         
 4   DD                   717 non-null    int64         
 5   ROL                  717 non-null    object        
 6   VICTIMA              717 non-null    object        
 7   SEXO                 717 non-null    object        
 8   EDAD                 717 non-null    object        
 9   FECHA_FALLECIMIENTO  717 non-null    object        
dtypes: datetime64[ns](1), int64(3), object(6)
memory usage: 56.1+ KB


Tras verificar el DataFrame, identificamos que los valores "SD" corresponden a "Sin Dato", por lo que se tomó la decisión de cambiar estos valores a NaN para estandarizar.

In [23]:
# Observamos la presencia de datos "SD" en el DataFrame
df_victimas.eq("SD").sum()

ID_hecho                0
FECHA                   0
AAAA                    0
MM                      0
DD                      0
ROL                    11
VICTIMA                 9
SEXO                    6
EDAD                   53
FECHA_FALLECIMIENTO    68
dtype: int64

In [24]:
# Llamamos a la función "analizar_valores_sd"
analizar_valores_sd(df_victimas)

,Columna,Cantidad de SD,Porcentaje de SD
5,ROL,11,1.534170
6,VICTIMA,9,1.255230
7,SEXO,6,0.836820
8,EDAD,53,7.391911
9,FECHA_FALLECIMIENTO,68,9.483961


In [25]:
# Reemplazamos "SD" por NaN en todo el DataFrame
df_victimas.replace(["SD","sd"], np.nan, inplace=True)

Preparamos los datos que usaremos al llamar a la función "data_cleaning".

In [26]:
# Columnas que pasaremos a minúscula
columns_to_lower = ["SEXO", "ROL"]

# Columnas que eliminaremos
columns_to_drop = ["FECHA", "AAAA", "MM", "DD", "FECHA_FALLECIMIENTO", "VICTIMA"]

# Columnas que renombraremos
rename_dict = {"ID_hecho":"id_siniestro", "SEXO":"sexo", "EDAD":"edad", "ROL":"rol"}

# Columnas que convertiremos a entero
columns_to_int = ["edad"]

# Columnas que agregaremos
new_columns_dict = {"gravedad":"fatal"}

In [27]:
# Columnas que categorizaremos
columns_to_categorize = ["sexo", "edad", "gravedad", "rol"]

In [28]:
# Llamamos a la función "data_cleaning"
df_victimas_cleaned = data_cleaning(df_victimas,
                                    strip_spaces=True, # Eliminar espacios en blanco
                                    lowercase_columns=columns_to_lower, # Convertir a minúsculas                                    
                                    drop_columns=columns_to_drop,  # Eliminar columnas
                                    rename_columns=rename_dict,  # Renombrar columnas 
                                    convert_to_int_columns=columns_to_int, #Conversión entero
                                    new_columns=new_columns_dict, # Agregar columnas
                                    )

In [29]:
# Llamamos nuevamente a la función "data_cleaning" para categorizar columnas
df_victimas_cleaned = data_cleaning(df_victimas_cleaned, categorize_columns=columns_to_categorize) 

In [30]:
# Mostramos el resultado
df_victimas_cleaned.head()

,id_siniestro,rol,sexo,edad,gravedad
0,2016-0001,conductor,masculino,19,fatal
1,2016-0002,conductor,masculino,70,fatal
2,2016-0003,conductor,masculino,30,fatal
3,2016-0004,conductor,masculino,18,fatal
4,2016-0005,conductor,masculino,29,fatal


### 4. Combinación de Datos

In [31]:
# Fusionamos los DataFrames
homicidios = pd.merge(df_hechos_cleaned, df_victimas_cleaned, left_on="id_siniestro", right_on="id_siniestro", how="inner")

In [32]:
# Mostramos el resultado
homicidios.head(5)

,id_siniestro,nro_victimas,fecha,franja_hora,tipo_calle,comuna,longitud,latitud,vehiculo_victima,vehiculo_acusado,rol,sexo,edad,gravedad
0,2016-0001,1,2016-01-01,4,avenida,8,-58.475340,-34.687570,moto,auto,conductor,masculino,19,fatal
1,2016-0002,1,2016-01-02,1,gral paz,9,-58.508775,-34.669777,auto,pasajeros,conductor,masculino,70,fatal
2,2016-0003,1,2016-01-03,7,avenida,1,-58.390403,-34.631894,moto,auto,conductor,masculino,30,fatal
3,2016-0004,1,2016-01-10,0,avenida,8,-58.465039,-34.680930,moto,NaN,conductor,masculino,18,fatal
4,2016-0005,1,2016-01-21,5,avenida,1,-58.387183,-34.622466,moto,pasajeros,conductor,masculino,29,fatal


### 5. Guardar el conjunto de datos limpio

In [33]:
# Los archivos se almacenan en local 
homicidios.to_csv("homicidios_cleaned.csv", index=False)